In [2]:
#import libraries
import requests
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd


In [3]:
# get events from n days ago
iso = 562 #Look in ACLED documentation ISO code for number this is for Niger
limit = 400

api_url = 'https://api.acleddata.com/acled/read?terms=accept&iso={}'.format(iso)
print (api_url, type(api_url)) #creates request according to ACLED format specifications - p. 13

https://api.acleddata.com/acled/read?terms=accept&amp;iso=562 &lt;class &#39;str&#39;&gt;


In [4]:
response = requests.get(api_url)
data = response.json()
data.keys()
data['count'] 

500

### From the documentation we know this is the max return --- How can we get all the results?

In [5]:
# Let's mkae a function that updates our search to get the new pages

def ping_acled(api_url): 
    '''
    Takes one parameter search term for API
    '''
    
    response = requests.get(api_url)
    data = response.json()
    return data
    

In [6]:

results = [] # empty data strcture to store results
num_results = 500 # condition to continue adding pages
count = 0  # tracker of results
page = 1 #Per the documentation each page will give us more results

while num_results == 500: #if less 500 or 0 we know we have all the results
    print ("starting ", page, " ", num_results) #just to see our progress
    api_url = 'https://api.acleddata.com/acled/read?terms=accept&iso={}&page={}'.format(iso,page) #the search
    data = ping_acled(api_url) #call the previous function 
    results.append(data['data']) #store in our results
    count += data['count'] #Track number of results
    num_results = data['count'] #update our condition
    page += 1 #update our page variable
    print ("Total Results ", count) #Track our progress
    



starting  1   500
Total Results  500
starting  2   500
Total Results  1000
starting  3   500
Total Results  1408


In [7]:
#Now I want to put them together into one giant result
super_list = []
for res in results: 
    super_list += res
    print (len(super_list))

500
1000
1408


In [8]:
#convert it into an pandas data frame or just use your data structure and do more stuff
niger_res = pd.DataFrame(super_list)
niger_res.head()

data_id  iso event_id_cnty event_id_no_cnty  event_date  year  \
0  7233612  562       NIR1460             1460  2020-09-23  2020   
1  7233305  562       NIR1459             1459  2020-09-22  2020   
2  7233937  562       NIR1458             1458  2020-09-22  2020   
3  7233304  562       NIR1457             1457  2020-09-21  2020   
4  7233611  562       NIR1456             1456  2020-09-21  2020   

  time_precision                  event_type                sub_event_type  \
0              1                     Battles                   Armed clash   
1              1      Strategic developments  Looting/property destruction   
2              1  Violence against civilians                        Attack   
3              1      Strategic developments  Looting/property destruction   
4              1                    Protests              Peaceful protest   

                                              actor1  ...    location  \
0  Boko Haram - Jamatu Ahli is-Sunnah lid-Dawatai...  ...       Alari   
1                   Unidentified Armed Group (Niger)  ...      Taroum   
2                   Unidentified Armed Group (Niger)  ...       Kossa   
3                   Unidentified Armed Group (Niger)  ...  Zigui Zeno   
4                                 Protesters (Niger)  ...       Diffa   

  latitude longitude geo_precision  \
0  14.4664   13.4416             1   
1  14.6744    2.2729             2   
2  14.6238    0.2657             1   
3  15.0237    2.4502             2   
4  13.3200   12.6100             1   

                                              source         source_scale  \
0                                     Jeunesse Diffa            New media   
1                       Undisclosed Source; Whatsapp  Local partner-Other   
2                                           Whatsapp            New media   
3  L&#39;Evenement Niger; Tadress24info; Urgence Till...   New media-National   
4                                     Jeunesse Diffa            New media   

                                               notes fatalities   timestamp  \
0  On 23 September 2020, presumed Boko Haram (JAS...          7  1601324148   
1  On 22 September 2020, unknown gunmen aboard mo...          0  1601324148   
2  On 22 September 2020, two unknown gunmen on a ...          1  1601324149   
3  On 21 September 2020, unknown gunmen seized an...          0  1601324147   
4  On 21 September 2020, a number of students sta...          0  1601324148   

  iso3  
0  NER  
1  NER  
2  NER  
3  NER  
4  NER  

[5 rows x 31 columns]

### Do the right thing, take some time to look at the codebook and see what these columns are

In [9]:
niger_res.columns


Index([&#39;data_id&#39;, &#39;iso&#39;, &#39;event_id_cnty&#39;, &#39;event_id_no_cnty&#39;, &#39;event_date&#39;,
       &#39;year&#39;, &#39;time_precision&#39;, &#39;event_type&#39;, &#39;sub_event_type&#39;, &#39;actor1&#39;,
       &#39;assoc_actor_1&#39;, &#39;inter1&#39;, &#39;actor2&#39;, &#39;assoc_actor_2&#39;, &#39;inter2&#39;,
       &#39;interaction&#39;, &#39;region&#39;, &#39;country&#39;, &#39;admin1&#39;, &#39;admin2&#39;, &#39;admin3&#39;,
       &#39;location&#39;, &#39;latitude&#39;, &#39;longitude&#39;, &#39;geo_precision&#39;, &#39;source&#39;,
       &#39;source_scale&#39;, &#39;notes&#39;, &#39;fatalities&#39;, &#39;timestamp&#39;, &#39;iso3&#39;],
      dtype=&#39;object&#39;)

### Homework --- Make a map of some ACLED Data (absolutely use the code from the Global Terrorism Database exercise)

In [10]:
# Let's take a look at TTP operations over time to create a "campaign progress map" to show the regions where TTP
# operational freedom expanded and contracted over time.
import pprint



In [20]:
# ATTENTION: This is the initial request.  For usability over sessions, I loaded the results into a JSON which I load in 2 cells from now.  To see the original output, remove the multiline comment and it will run.

# For the sake of practicality, we'll limit our countries to AFG/PAK/IND/BNG
countries = [4, 586, 356, 50]


# My gut tells me that I can get paginated returns from all countries accross the entire range of data without crushing
# my application to start, but let's test that.
url_base = 'https://api.acleddata.com/acled/read?terms=accept&iso={}&page={}'
failures = []
results = []
for country in countries:
    page = 1
    last_results = 500
    while last_results == 500:
        # request the current page for the current country
        if page % 10 == 1:
            print('Requesting page {} of country {}'.format(page, country))
        url = url_base.format(country, page)
        response = requests.get(url)
        # make sure the request succeeded; otherwise "log" the failure
        if (200 <= response.status_code < 300):
            page_data = response.json()
            # update last_results
            last_results = page_data['count']
            # iterate page upward for the next request
            page += 1
            # store this page in results
            results.append(page_data['data'])
        else:
            failure = {
                'message': 'Request for page {} of country {} failed with response {}'.format(page, country, response.status_code),
                'data': 'response.json()'
            }
            print(failure['message'])
            failures.append(failure)
        # If my intution is borked, save me from being an idiot
        if page > 1000:
            failure = {'message': '*** 1000 pages of events reached for country {} ***'.format(country)}
            print(failure['message'])
            failures.append(failure)
            break
    print('Country {} complete; total pages: {}'.format(country, page))
if len(failures) == 0:
    print('Request completed without failures!')
else:
    print('Failures detected!')



Requesting page 1 of country 4
Requesting page 11 of country 4
Requesting page 21 of country 4
Requesting page 31 of country 4
Requesting page 41 of country 4
Requesting page 51 of country 4
Requesting page 61 of country 4
Requesting page 71 of country 4
Requesting page 81 of country 4
Requesting page 91 of country 4
Country 4 complete; total pages: 95
Requesting page 1 of country 586
Requesting page 11 of country 586
Requesting page 21 of country 586
Requesting page 31 of country 586
Requesting page 41 of country 586
Requesting page 51 of country 586
Requesting page 61 of country 586
Requesting page 71 of country 586
Requesting page 81 of country 586
Requesting page 91 of country 586
Requesting page 101 of country 586
Requesting page 111 of country 586
Requesting page 121 of country 586
Country 586 complete; total pages: 123
Requesting page 1 of country 356
Requesting page 11 of country 356
Requesting page 21 of country 356
Requesting page 31 of country 356
Requesting page 41 of count

In [21]:
# Because I don't want to have to re-run that request if the notebook crashes, I'm going to write the data to a json.
import json

# Compile the entries into a single list
events = []
for page in results:
    events += page

# Place into a well-formatted JSON structure, then write to file
events_dict = {'events': events}
with open('events.json', 'w') as events_json:
    json.dump(events_dict, events_json)


In [22]:
#Fun fact, I actually crashed out my editor by trying to load the 188MB file as a file.  Glad I saved it.  The code here will load the json into the events variable, if you want to skip the API calls.
import json

'''events = []
with open('events.json') as events_json:
    events_dict = json.load(events_json)
    events += events_dict['events']'''

print(len(events))

206334


In [23]:
# Load into pandas and start cleaning
import re
events_frame = ''
events_frame = pd.DataFrame(events)
actors = events_frame.actor1.unique()
actors

array([&#39;Taliban&#39;, &#39;Military Forces of Afghanistan (2014-)&#39;,
       &#39;Police Forces of Afghanistan (2014-)&#39;, ...,
       &#39;Kastol Communal Militia (Bangladesh)&#39;,
       &#39;Garmati Communal Militia (Bangladesh)&#39;,
       &#39;Kalaruka Communal Militia (Bangladesh)&#39;], dtype=object)

In [25]:
# Do some initial data exploration.
print(events_frame.event_date.min())

len(actors)

2010-01-01


2327

In [26]:
# Alright, let's narrow down those 2327 actors. Something "pak," something "taliban."

scrubbed = (events_frame[
    events_frame.actor1.str.contains(r'pak', case=False) & 
    events_frame.actor1.str.contains(r't.l.b.n', case=False)])
scrubbed_actors = scrubbed.actor1.unique()
print(len(scrubbed_actors))
print(len(scrubbed))
scrubbed_actors

15
1916


array([&#39;TTP: Tehreek-i-Taliban Pakistan&#39;,
       &#39;TTP-S: Tehreek-i-Taliban Pakistan-Sajna&#39;,
       &#39;TTP-HGB: Tehreek-i-Taliban Pakistan-Hafiz Gul Bahadur&#39;,
       &#39;TTP-MS: Tehreek-i-Taliban Pakistan-Mufti Shakir&#39;,
       &#39;TTP-MN: Tehreek-i-Taliban Pakistan-Mullah Nazir&#39;,
       &#39;TTP-Mucchar: Tehreek-i-Taliban Pakistan-Mucchar&#39;,
       &#39;TTP-Mohmand: Tehreek-i-Taliban Pakistan-Mohmand&#39;,
       &#39;TTP-Mehsud: Tehreek-i-Taliban Pakistan-Mehsud&#39;,
       &#39;TTP-TA: Tehreek-i-Taliban Pakistan-Tariq Afridi&#39;,
       &#39;TTP-OA: Tehreek-i-Taliban Pakistan-Omar Afridi&#39;,
       &#39;TTP-HK: Tehreek-i-Taliban Pakistan-Hafiz Kochwan&#39;,
       &#39;TTP-O: Tehreek-i-Taliban Pakistan-Orakzai&#39;,
       &#39;TTIP: Tehreek-i-Taliban Islami Pakistan&#39;,
       &#39;TTP-DH: Tehreek-i-Taliban Pakistan-Daulat Hafiz&#39;,
       &#39;TTP-T: Tehreek-i-Taliban Pakistan-Toofan&#39;], dtype=object)

In [27]:
#OKAY! Well, that looks good.  Except for the part where it's too good.  What am I missing?

scrubbed_testing = (events_frame[
    events_frame.actor1.str.contains(r'p.*k', case=False) & 
    events_frame.actor1.str.contains(r't.*l.*b', case=False)])
scrubbed_testing_actors = scrubbed_testing.actor1.unique()
print(len(scrubbed_testing_actors))
scrubbed_testing_actors

28


array([&#39;TTP: Tehreek-i-Taliban Pakistan&#39;,
       &#39;TTP-S: Tehreek-i-Taliban Pakistan-Sajna&#39;,
       &#39;TTP-HGB: Tehreek-i-Taliban Pakistan-Hafiz Gul Bahadur&#39;,
       &#39;TLP: Tehreek-e-Labbaik Pakistan&#39;,
       &#39;Kotli Babay Heerian Communal Militia (Pakistan)&#39;,
       &#39;Fateh Khankhel Tribal Militia (Pakistan)&#39;,
       &#39;Utmankhel Tribal Militia (Pakistan)&#39;,
       &#39;TTP-MS: Tehreek-i-Taliban Pakistan-Mufti Shakir&#39;,
       &#39;TTP-MN: Tehreek-i-Taliban Pakistan-Mullah Nazir&#39;,
       &#39;TTP-Mucchar: Tehreek-i-Taliban Pakistan-Mucchar&#39;,
       &#39;Kotla Bakht Ali Communal Militia (Pakistan)&#39;,
       &#39;TTP-Mohmand: Tehreek-i-Taliban Pakistan-Mohmand&#39;,
       &#39;TTP-Mehsud: Tehreek-i-Taliban Pakistan-Mehsud&#39;,
       &#39;TTP-TA: Tehreek-i-Taliban Pakistan-Tariq Afridi&#39;,
       &#39;TTP-OA: Tehreek-i-Taliban Pakistan-Omar Afridi&#39;,
       &#39;TTP-HK: Tehreek-i-Taliban Pakistan-Hafiz Kochwan&#39;,
   

In [28]:
# Well that's promising, at least.  If this was something other than an overdue homework, I might sort all actors alphabetically and remove the TTP variants I've found, but yeah, overdue homework.  This is good enough, and it's still 1,916 events.

# Let's start by removing events that are missing geodata or have precision rank 3 (which is too imprecise and will dump event into regional capitals.)
import numpy as np
scrubbed_testing = scrubbed[scrubbed['geo_precision'].astype(np.int32, copy=True) < 3]
scrubbed_testing.latitude.dropna
scrubbed_testing.longitude.dropna

&lt;bound method Series.dropna of 26176     70.8657
33478     71.2717
47096     70.0698
47304     70.2785
47327     70.2785
           ...   
107584    70.5083
107587    69.8940
107596    66.4681
107597    70.8511
107600    71.5141
Name: longitude, Length: 1906, dtype: object&gt;

In [29]:
scrubbed_testing

data_id  iso event_id_cnty event_id_no_cnty  event_date  year  \
26176   6836320    4      AFG11027            11027  2018-06-29  2018   
33478   6843620    4       AFG7959             7959  2017-12-27  2017   
47096   7230501  586      PAK61089            61089  2020-09-12  2020   
47304   7220596  586      PAK60989            60989  2020-09-07  2020   
47327   7220937  586      PAK61070            61070  2020-09-06  2020   
...         ...  ...           ...              ...         ...   ...   
107584  4965985  586         PAK21               21  2010-01-04  2010   
107587  4966012  586         PAK18               18  2010-01-04  2010   
107596  4966090  586          PAK4                4  2010-01-01  2010   
107597  4966091  586          PAK8                8  2010-01-01  2010   
107600  4966121  586          PAK7                7  2010-01-01  2010   

       time_precision                  event_type  \
26176               2                     Battles   
33478               1      Strategic developments   
47096               1  Explosions/Remote violence   
47304               1                     Battles   
47327               1                     Battles   
...               ...                         ...   
107584              1                     Battles   
107587              1  Explosions/Remote violence   
107596              1      Strategic developments   
107597              1  Explosions/Remote violence   
107600              1  Explosions/Remote violence   

                           sub_event_type                           actor1  \
26176                         Armed clash  TTP: Tehreek-i-Taliban Pakistan   
33478               Disrupted weapons use  TTP: Tehreek-i-Taliban Pakistan   
47096       Remote explosive/landmine/IED  TTP: Tehreek-i-Taliban Pakistan   
47304                         Armed clash  TTP: Tehreek-i-Taliban Pakistan   
47327                         Armed clash  TTP: Tehreek-i-Taliban Pakistan   
...                                   ...                              ...   
107584                        Armed clash  TTP: Tehreek-i-Taliban Pakistan   
107587  Shelling/artillery/missile attack  TTP: Tehreek-i-Taliban Pakistan   
107596       Looting/property destruction  TTP: Tehreek-i-Taliban Pakistan   
107597                       Suicide bomb  TTP: Tehreek-i-Taliban Pakistan   
107600      Remote explosive/landmine/IED  TTP: Tehreek-i-Taliban Pakistan   

        ...            location latitude longitude geo_precision  \
26176   ...        Mohmand Dara  34.2553   70.8657             2   
33478   ...  Shaigal Aw Shiltan  34.9692   71.2717             2   
47096   ...           Miranshah  33.0001   70.0698             2   
47304   ...             Mir Ali  32.9705   70.2785             1   
47327   ...             Mir Ali  32.9705   70.2785             2   
...     ...                 ...      ...       ...           ...   
107584  ...            Janikhel  32.8011   70.5083             1   
107587  ...        Metari Narai  32.6329   69.8940             1   
107596  ...              Chaman  30.8944   66.4681             2   
107597  ...        Lakki Marwat  32.5992   70.8511             1   
107600  ...            Salarzai  34.9256   71.5141             1   

                                   source source_scale  \
26176   Bakhtar News Agency; Khaama Press     National   
33478                        Khaama Press     National   
47096     Daily Regional Times (Pakistan)     National   
47304              Daily Times (Pakistan)     National   
47327         South Asia Terrorism Portal        Other   
...                                   ...          ...   
107584                  Right Vision News     National   
107587                  Right Vision News     National   
107596                  Right Vision News     National   
107597                  Right Vision News     National   
107600                  Right Vision News     National   

                                           

In [30]:
# Now let's make sure the events are operational in nature based on their event_type
scrubbed = scrubbed_testing

event_types = scrubbed.event_type.unique()
len(event_types)
event_types

array([&#39;Battles&#39;, &#39;Strategic developments&#39;, &#39;Explosions/Remote violence&#39;,
       &#39;Violence against civilians&#39;, &#39;Riots&#39;], dtype=object)

In [34]:
# I'm immediately suspicious of "Riots" and "Strategic Developments"; what events are those?
suspect_events = scrubbed[scrubbed['event_type'].isin(['Strategic developments', 'Riots'])]
print(len(suspect_events))
pprint.pprint([[x, ''] for x in list(suspect_events['notes'])], indent=1, width=80)

37
[[&#39;Detonation: A senior leader of the TTP was accidentally killed by his own &#39;
  &#39;explosives in Shaigal Aw Shiltan District, Kunar Province, on December 27, &#39;
  &#39;2017, while militants were busy planting roadside mines. Other local &#39;
  &#39;reports claim that he was shot by unknown gunmen.&#39;,
  &#39;&#39;],
 [&#39;Defusal: On December 30, the Bomb Disposal Squad defused an IED mounted on &#39;
  &#39;a stolen motorcycle by alleged members of Mufti Shakir group in PIB Colony &#39;
  &#39;of Gulshan town (Karachi, Sindh).&#39;,
  &#39;&#39;],
 [&#39;Property destruction: On January 18, the Government College at Wana, SWA, &#39;
  &#39;was burned down by suspected TTP militants.&#39;,
  &#39;&#39;],
 [&#39;Arrests: Counter Terrorism Department Hazara Division arrested a TTP &#39;
  &#39;commander involved in number of terror activities in Hazara Division.&#39;,
  &#39;&#39;],
 [&#39;Agreement: On June 19, former Tehreek-e-Taliban Pakistan chief Hakimullah &#39

In [36]:
#Looking at that, I'm going to remove references in those event types to arrests, agreements, coalition forming, and reports that leadership have moved, since those don't represent operational activities

good_event_types = scrubbed[~scrubbed['event_type'].isin(['Strategic developments', 'Riots'])]
exception_events = scrubbed[scrubbed['event_type'].isin(['Strategic developments', 'Riots'])]
exception_events = (exception_events[
    ~exception_events.notes.str.contains(r'^Arrests') & 
    ~exception_events.notes.str.contains(r'^Agreement') & 
    ~exception_events.notes.str.contains(r'formed a joint lashkar') & 
    ~exception_events.notes.str.contains(r'were arrested') & 
    ~exception_events.notes.str.contains(r'leadership moved to')
    ])

print(len(exception_events))
pprint.pprint([[x, ''] for x in list(exception_events['notes'])], indent=1, width=80)

22
[[&#39;Detonation: A senior leader of the TTP was accidentally killed by his own &#39;
  &#39;explosives in Shaigal Aw Shiltan District, Kunar Province, on December 27, &#39;
  &#39;2017, while militants were busy planting roadside mines. Other local &#39;
  &#39;reports claim that he was shot by unknown gunmen.&#39;,
  &#39;&#39;],
 [&#39;Defusal: On December 30, the Bomb Disposal Squad defused an IED mounted on &#39;
  &#39;a stolen motorcycle by alleged members of Mufti Shakir group in PIB Colony &#39;
  &#39;of Gulshan town (Karachi, Sindh).&#39;,
  &#39;&#39;],
 [&#39;Property destruction: On January 18, the Government College at Wana, SWA, &#39;
  &#39;was burned down by suspected TTP militants.&#39;,
  &#39;&#39;],
 [&#39;Property destruction: On November 30, suspected TTP militants set fire to a &#39;
  &#39;cell tower in the Gurgaz Gagezai area (Mohmand Agency, FATA).&#39;,
  &#39;&#39;],
 [&#39;Property destruction: On October 17, Pakistani militants operating in &#39;
  &

In [37]:
event_scrubbed = pd.concat([good_event_types, exception_events])
event_scrubbed

data_id  iso event_id_cnty event_id_no_cnty  event_date  year  \
26176   6836320    4      AFG11027            11027  2018-06-29  2018   
47096   7230501  586      PAK61089            61089  2020-09-12  2020   
47304   7220596  586      PAK60989            60989  2020-09-07  2020   
47327   7220937  586      PAK61070            61070  2020-09-06  2020   
47347   7220430  586      PAK61032            61032  2020-09-05  2020   
...         ...  ...           ...              ...         ...   ...   
105923  7137744  586       PAK1636             1636  2010-03-31  2010   
105958  4961770  586       PAK1577             1577  2010-03-29  2010   
106590  4963161  586        PAK959              959  2010-03-03  2010   
107327  7028446  586        PAK277              277  2010-01-24  2010   
107596  4966090  586          PAK4                4  2010-01-01  2010   

       time_precision                  event_type  \
26176               2                     Battles   
47096               1  Explosions/Remote violence   
47304               1                     Battles   
47327               1                     Battles   
47347               1                     Battles   
...               ...                         ...   
105923              1      Strategic developments   
105958              1      Strategic developments   
106590              1      Strategic developments   
107327              1      Strategic developments   
107596              1      Strategic developments   

                           sub_event_type                           actor1  \
26176                         Armed clash  TTP: Tehreek-i-Taliban Pakistan   
47096       Remote explosive/landmine/IED  TTP: Tehreek-i-Taliban Pakistan   
47304                         Armed clash  TTP: Tehreek-i-Taliban Pakistan   
47327                         Armed clash  TTP: Tehreek-i-Taliban Pakistan   
47347                         Armed clash  TTP: Tehreek-i-Taliban Pakistan   
...                                   ...                              ...   
105923       Looting/property destruction  TTP: Tehreek-i-Taliban Pakistan   
105958  Non-violent transfer of territory  TTP: Tehreek-i-Taliban Pakistan   
106590   Headquarters or base established  TTP: Tehreek-i-Taliban Pakistan   
107327              Disrupted weapons use  TTP: Tehreek-i-Taliban Pakistan   
107596       Looting/property destruction  TTP: Tehreek-i-Taliban Pakistan   

        ...      location latitude longitude geo_precision  \
26176   ...  Mohmand Dara  34.2553   70.8657             2   
47096   ...     Miranshah  33.0001   70.0698             2   
47304   ...       Mir Ali  32.9705   70.2785             1   
47327   ...       Mir Ali  32.9705   70.2785             2   
47347   ...      Sarwakai  32.0581   69.3090             1   
...     ...           ...      ...       ...           ...   
105923  ...    Utman Khel  32.2253   69.6549             2   
105958  ...        Kalaya  33.7480   70.9629             2   
106590  ...        Kalaya  33.7480   70.9629             2   
107327  ...     Miranshah  33.0001   70.0698             2   
107596  ...        Chaman  30.8944   66.4681             2   

                                              source    source_scale  \
26176              Bakhtar News Agency; Khaama Press        National   
47096                Daily Regional Times (Pakistan)        National   
47304                         Daily Times (Pakistan)        National   
47327                    South Asia Terrorism Portal           Other   
47347   South Asia Terrorism Portal; Dawn (Pakistan)  Other-National   
...                                              ...             ...   
105923                             Right Vision News        National   
105958                             Right Vision News        National   
106590                             Right Vision News        National   
107327                             Right Vision News        National   
107596              

In [38]:
# Basic data to make sure nothing's too crazy

scrubbed = event_scrubbed
print(scrubbed.iso3.unique())

[&#39;AFG&#39; &#39;PAK&#39;]


In [42]:
# Start the mapping piece.


import math #this is used in graphic section to use the irrational number pi
from bokeh.plotting import figure, output_notebook, show #builds interactive graphs for python
from bokeh.models import Range1d
from bokeh.tile_providers import get_provider, Vendors
from pyproj import Transformer

tile_provider = get_provider('STAMEN_TERRAIN')
output_notebook() #Allows inline plotting for Juptyer notebook

Loading BokehJS ...

In [61]:
# Pare data down to map-relevant information

ttp_map = scrubbed[['data_id', 'latitude', 'longitude']]
ttp_map.head(10)

data_id latitude longitude
26176  6836320  34.2553   70.8657
47096  7230501  33.0001   70.0698
47304  7220596  32.9705   70.2785
47327  7220937  32.9705   70.2785
47347  7220430  32.0581   69.3090
47419  7197596  32.7138   70.0000
47488  7199613  34.8281   71.8408
47509  7198298  32.3011   69.5799
48053  7197731  32.5680   69.8291
48111  7198427  34.8236   71.6855

In [87]:
# Generate the bounding box and convert coordinates
import numpy as np

map_bounds = [(23.25,79.50), (39.00,60.00)]

transformer = Transformer.from_crs('epsg:4326','epsg:3857')
bounding_box = []
for pt in transformer.itransform(map_bounds): 
    bounding_box.append(pt)
ll_points = list(zip(ttp_map['latitude'].astype(float).tolist(), ttp_map['longitude'].astype(float).tolist()))
wm_points = list(transformer.itransform(ll_points))
ttp_map['wm_latitude'] = [x[1] for x in wm_points]
ttp_map['wm_longitude'] = [x[0] for x in wm_points]
ttp_map.head()

data_id latitude longitude   wm_latitude  wm_longitude
26176  6836320  34.2553   70.8657  4.063134e+06  7.888734e+06
47096  7230501  33.0001   70.0698  3.895317e+06  7.800134e+06
47304  7220596  32.9705   70.2785  3.891389e+06  7.823367e+06
47327  7220937  32.9705   70.2785  3.891389e+06  7.823367e+06
47347  7220430  32.0581   69.3090  3.770940e+06  7.715443e+06

In [88]:
# Plots the bounding box
p = figure(
    x_range = (bounding_box[0][0], bounding_box[1][0]),
    y_range = (bounding_box[0][1], bounding_box[1][1]),
    x_axis_type = "mercator",
    y_axis_type = "mercator")
# add the map form the Bokeh map vendor in this case Stamen_Terrain --- see documentation
p.add_tile(tile_provider)
# Places a circle for each converted lat/long attack 
for index, row in ttp_map.iterrows():
    p.circle(x = row['wm_longitude'], y = row['wm_latitude'], color = 'cyan')

# shows the plot
show(p)